![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# <center> THIS RESEARCH NOTEBOOK IS FOR DOCUMENTATION PURPOSE ONLY <center>

In [1]:
%matplotlib inline
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Jupyter")
AddReference("QuantConnect.Indicators")
from System import *
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar, QuoteBar
from QuantConnect.Jupyter import *
from QuantConnect.Indicators import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# create an instance
qb = QuantBook()
plt.style.use('seaborn-whitegrid')

In [2]:
### user-defined inputs --------------------------------------------------------------

# add tickers for risky assets for momentum asset allocation
riskyTickers = ['SPY',   # US equities
                'VGK',  # European equities
                'EWJ',  # Japanese equities
                'EEM',  # Emerging market equities
                'VNQ',  # US REITs
                'RWX',  # International REITs
                'TLT',  # US 30-year Treasuries
                'DBC',  # Commodities
                'GLD',  # Gold
                ]

# number of top momentum securities to keep  
topMomentum = 5

### -----------------------------------------------------------------------------------

symbolsDict = {}

# adding data
for ticker in riskyTickers:
    symbolsDict[ticker] = qb.AddEquity(ticker)

# get the symbols
activeSymbols = [symbolsDict[ticker].Symbol for ticker in riskyTickers]

# get historical data
if len(activeSymbols) > 0:
    history = qb.History(activeSymbols, 253, Resolution.Daily)

# visualize data
history.head()

### THIS CLASS CONTAINS THE RETURN SERIES AND CALCULATION OF MOMENTUM SCORE FOR EACH SYMBOL

In [3]:
class SymbolData:
    
    ''' Contain data specific to a symbol required by this model '''
    
    def __init__(self, symbol):
        
        self.Symbol = symbol

    def CalculateMomentumScore(self, history):
        
        ''' Calculate the weighted average momentum value for each security '''
        
        self.returnSeries = history.loc[str(self.Symbol)]['close'].pct_change(periods = 1).dropna() # 1-day returns for last year
        
        cumRet1 = (self.returnSeries.tail(21).add(1).prod()) - 1 # 1-month momentum
        cumRet3 = (self.returnSeries.tail(63).add(1).prod()) - 1 # 3-month momentum
        cumRet6 = (self.returnSeries.tail(126).add(1).prod()) - 1 # 6-month momentum
        cumRet12 = (self.returnSeries.tail(252).add(1).prod()) - 1 # 12-month momentum
        
        self.momentumScore = (cumRet1 * 12 + cumRet3 * 4 + cumRet6 * 2 + cumRet12) # weighted average momentum

### HOW WE GET TO CUMULATIVE RETURNS - VISUALIZING AN EXAMPLE WITH SPY FOR 1 MONTH

* First we get the close prices for the last 22 trading days (to get 21 returns)
* Then we calculate the percent change of those last 22 close prices to get the last 21 returns
* Finally, we calculate the cumulative return over those 21 daily returns

In [4]:
# get the close prices for the last 22 trading days (in order to later get 21 returns)
history.loc[str(symbolsDict['SPY'])]['close'].tail(22)

In [5]:
# get the last 21 returns
SPYreturnSeries = history.loc[str(symbolsDict['SPY'])]['close'].pct_change(periods = 1).dropna() # 1-day returns
SPYreturnSeries.tail(21)

In [6]:
# we simply calculate the cumulative return from the last 21 returns
SPYcumRet = (SPYreturnSeries.tail(21).add(1).prod()) - 1
SPYcumRet

### PROCESS TO CALCULATE THE TOP SECURITIES BASED ON DUAL MOMENTUM

* Apply the CalculateMomentumScore function (from SymbolData Class) to calculate the momentum score of each asset using the formula **momentumScore = (1-Month CumumlativeReturns * 12 + 3-Month CumumlativeReturns * 4 + 6-Month CumumlativeReturns * 2 + 12-Month CumumlativeReturns)**
* Apply **Absolute Momentum** => Keep only securities with Positive Absolute Momentum
* Apply **Relative Momentum** => Keep the Top N securities with highest Momentum Score

In [7]:
# dictionary to store calculations
calculations = {}

# loop through each ticker to calculate the momentum score
for symbol in activeSymbols:

    if (str(symbol) not in history.index
    or history.loc[str(symbol)].get('close') is None
    or history.loc[str(symbol)].get('close').isna().any()):
        calculations.pop(symbol, None)
        continue
    else:
        # add symbol to calculations
        calculations[symbol] = SymbolData(symbol)
        try:
            # get momentum score
            calculations[symbol].CalculateMomentumScore(history)
        except Exception:
            calculations.pop(symbol, None)
            continue

# dataframe to visualize data
dfCalculations = pd.DataFrame([(x.Symbol.Value, x.momentumScore) for x in calculations.values()],
                                           columns = ['ticker', 'momentum_score'])

dfCalculations = dfCalculations.sort_values(by = ['momentum_score'], ascending = False)
dfCalculations

In [8]:
# plot a bar chart with momentum scores
dfCalculations.plot.bar(title = 'Momentum Score by Ticker', x = 'ticker', y = 'momentum_score', rot = 0, legend = False)

In [9]:
# filter applying Absolute Momentum: get the securities with positive momentum
positiveMomentumSecurities = list(filter(lambda x: x.momentumScore > 0 and x.Symbol in activeSymbols, calculations.values()))

dfpositiveMomentumSecurities = pd.DataFrame([(x.Symbol.Value, x.momentumScore) for x in positiveMomentumSecurities],
                                            columns = ['ticker', 'momentum_score'])
dfpositiveMomentumSecurities

In [10]:
# perform Relative Momentum: sort descending by momentum score and select the top n
topMomentumSecurities = sorted(positiveMomentumSecurities, key = lambda x: x.momentumScore, reverse = True)[:topMomentum]

dftopMomentumSecurities = pd.DataFrame([(x.Symbol.Value, x.momentumScore) for x in topMomentumSecurities],
                                            columns = ['ticker', 'momentum_score'])

dftopMomentumSecurities = dftopMomentumSecurities.sort_values(by = ['momentum_score'], ascending = False)
dftopMomentumSecurities

In [11]:
# plot a bar chart with the top securities and their momentum score
dftopMomentumSecurities.plot.bar(title = 'Top Securities - Momentum Score by Ticker', x = 'ticker', y = 'momentum_score', rot = 0, legend = False)

### HOW TO CALCULATE THE COVARIANCE MATRIX USING A CUSTOM CORRELATION MATRIX

This strategy applies portfolio optimization to calculate the optimal weights for each security in the portfolio. However, it uses a different Correlation Matrix based on the 1-3-6-12 momentum weighting.
Below we show how this Correlation Matrix is calculated in the algorithm and especially how we can generate the Covariance Matrix needed for the optimization process.

* First we create a DataFrame with **log-returns** from the top momentum securities
* Secondly we calculate **Correlation Matrix = 1-Month CorrelationMatrix * 12 + 3-Month CorrelationMatrix * 4 + 6-Month CorrelationMatrix * 2 + 12-Month CorrelationMatrix ) / 19**
* Then create the **Standard Deviation Matrix = 1-Month Standard Deviation of Returns**
* Get the **Transpose Standard Deviation Matrix**
* Compute **Volatility Matrix = Dot product between Standard Deviation Matrix and Transpose Standard Deviation Matrix**
* And final **Covariance Matrix = Element-wise multiplication (Hadamard product) of CorrelationMatrix and Volatility Matrix**

In [12]:
# create a dictionary keyed by the symbols in calculations with a pandas.Series as value to create a data frame of log-returns
logReturnsDict = { str(symbol): np.log(1 + symbolData.returnSeries) for symbol, symbolData in calculations.items()
                  if symbol.Value in dftopMomentumSecurities['ticker'].values}
logReturnsDf = pd.DataFrame(logReturnsDict)
logReturnsDf.tail()

In [13]:
# create correlation matrix with 1-3-6-12 momentum weighting
corrMatrix = ( logReturnsDf.tail(21).corr() * 12 + logReturnsDf.tail(63).corr() * 4 + logReturnsDf.tail(126).corr() * 2 + logReturnsDf.tail(252).corr() ) / 19
corrMatrix

In [14]:
# create standard deviation matrix using the 1-month standard deviation of returns
stdMatrix = pd.DataFrame(logReturnsDf.tail(21).std()) # column vector (one row per symbol and one single column with the standard deviation)
stdMatrix

In [77]:
# get its transpose
stdMatrixTranspose = stdMatrix.T # row vector (one single row with the standard deviation and one column per symbol)
stdMatrixTranspose

In [16]:
# compute the dot product between stdMatrix and its transpose to get the volatility matrix
volMatrix = stdMatrix.dot(stdMatrixTranspose) # square NxN matrix with the variances of each symbol on the diagonal and the product of stds on the off diagonal
volMatrix

In [17]:
# calculate the covariance matrix by doing element-wise multiplication of correlation matrix and volatility matrix
covMatrix = corrMatrix.multiply(volMatrix)
covMatrix